In [1]:
import pandas as pd
from bs4 import BeautifulSoup
import requests
import json
import re
import time
import pandas as pd
from tqdm import tqdm

In [2]:
from faker import Faker
fake = Faker(locale='ru')

In [3]:
headers = {
    'User-Agent': fake.chrome(),
    'accept-language': 'en-US,en;q=0.9',
    'pragma': 'np-cache',
    'referer': 'http://manufacturers.ru/companies'
}

In [4]:
MAIN_URL = "http://manufacturers.ru"
COMPANIES_URL = "http://manufacturers.ru/companies"

In [5]:
MAX_PAGES = 1260

In [6]:
companies_urls = []
for i in tqdm(range(MAX_PAGES)):
    params = {'page': i}
    companies_answer = requests.get(COMPANIES_URL, headers=headers, params=params)
    soup = BeautifulSoup(companies_answer.text)
    companies = soup.find_all('div', {'class': 'sim-item'})
    for c in companies:
        try:
            companies_urls.append(MAIN_URL + c['about'])
        except KeyError:
            pass
    time.sleep(0.1) # sleep for 100 ms

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1260/1260 [28:57<00:00,  1.38s/it]


In [12]:
def build_dict_from_block(rows):
    if rows is None:
        return dict()
    tr_rows = rows.find_all('tr')
    tuples = []
    for r in tr_rows:
        tuples.append([re.sub("\s\s+", " ", td.text.replace("\n", "").strip()) for td in r.findChildren('td')])
    return dict((x, y) for x, y in tuples)

In [20]:
total_dicts = []
for c_url in tqdm(companies_urls):
    c_info = requests.get(c_url, headers=headers)
    soup = BeautifulSoup(c_info.text)

    brand_info = soup.find('div', {'class': 'l-brand-block'})
    image_block = brand_info.find('img')

    all_info = soup.find('div', {'class': 'company-right-wrap'})
    about = all_info.find(id='about')

    description = {
        'Название': about.find('h1').text,
        'Описание': " ".join([p.text for p in about.find_all('p')]),
        'Источник': c_url
    }

    if image_block is not None:
        description['Brand-Image'] = MAIN_URL + image_block['src']

    category_block = all_info.find('div', id='category')
    categories = build_dict_from_block(category_block)

    requisites_block = all_info.find('div', id='company')
    requisites = build_dict_from_block(requisites_block)

    contacts_block = all_info.find('tbody', {'class': 'cont-tbody'})
    contacts = build_dict_from_block(contacts_block)

    super_dict = {}
    for d in [description, categories, requisites, contacts]:
        for k, v in d.items():
            super_dict[k] = v

    time.sleep(0.1) # sleep for 100 ms

    total_dicts.append(super_dict)

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████| 12600/12600 [1:53:41<00:00,  1.85it/s]


In [23]:
companies_df = pd.DataFrame(total_dicts)
companies_df.sample(10)

,Название,Описание,Источник,Brand-Image,Отрасль,Категория,Подкатегория,ИНН,ОГРН,КПП,Адрес,Телефоны,Почта,Сайт,Вконтакте,Год основания,Количество сотрудников,Название юридического лица,Дата регистрации,YouTube
12412,Агрофирма Лето,Основное направление деятельности группы компа...,http://manufacturers.ru/company/agrofirma-leto,http://manufacturers.ru/sites/default/files/st...,Сельское хозяйство,Растениеводство,Овощеводство,2311281809,1182375116373,231101001,"350921, Краснодарский край, город Краснодар, п...",+ 7 918 675 00 01 (отдел продаж) + 7 938 404 3...,a23leto@mail.ru,http://agroleto.ru,NaN,NaN,1,"Агрофирма Лето, ООО",14.12.2018,NaN
12387,АгроСевТорг,Компания «АгроСевТорг» основана в 2012 году. О...,http://manufacturers.ru/company/agrosevtorg,http://manufacturers.ru/sites/default/files/st...,Пищевая промышленность,Бакалейное производство,Масложировая промышленность,2223589513,1122223012394,222501001,"656003, Алтайский край, город Барнаул, улица М...",8 (3852) 50-62-40,agrosevtorg@mail.ru,http://agrosevtorg.ru,NaN,NaN,14,"Аст, ООО",08.10.2012,NaN
3408,Стерх,ООО «СТЕРХ» - крупнейший производитель пластик...,http://manufacturers.ru/company/sterkh,http://manufacturers.ru/sites/default/files/st...,Химическая промышленность,Органическое (нефтехимическое) производство,Производство изделий из полимеров,6163134594,1146195003140,614101001,"Батайск, Ольгинский тупик, 29Б",+7 800 550-62-01,sterh-pf@mail.ru,http://sterh-pf.ru,NaN,NaN,16,"Стерх, ООО",21.03.2014,NaN
8297,Колнаг,ООО «Колнаг» российский производитель и постав...,http://manufacturers.ru/company/kolnag,http://manufacturers.ru/sites/default/files/st...,Машиностроение,Сельскохозяйственное машиностроение,Производство почвообрабатывающего оборудования...,5022049930,1165022050038,502201001,"г. Коломна, улица Астахова, дом 4",8-496-610-03-83,info@kolnag.ru,http://kolnag.ru,NaN,NaN,88,"Колнаг, ООО",12.01.2016,NaN
5411,Политранс,Компания «Политранс» - один из ведущих российс...,http://manufacturers.ru/company/politrans,http://manufacturers.ru/sites/default/files/st...,Транспортное машиностроение,Производство техники специального назначения,"Производство дорожно-строительной техники, Обс...",7453062150,1027403862815,745301001,"Челябинск, ул. Труда, д. 78, оф. 806",7 (351) 247-46-10 (Отдел сервиса и продаж запа...,info@politrans.ru (генеральный директор),http://politrans.ru,NaN,NaN,428,"ПКФ ""Политранс"", ООО",08.10.2002,NaN
5005,Кредо,Деятельность компании «Кредо» направлена на ра...,http://manufacturers.ru/company/kredo,http://manufacturers.ru/sites/default/files/st...,"Горная промышленность, Лесная промышленность","Угольная промышленность, Лесохимическая промыш...","Производство угольных брикетов, Производство д...",7701763902,1077764819043,770701001,"г. Москва, ул. Петровка, 17, строение 4",8 800 700 49 52,info@rdc-credo.ru,http://rdc-credo.ru,NaN,NaN,9,"НПК ""КРЕДО"", ООО",29.12.2007,NaN
1595,Вторресурс,Наша компания предлагает на выгодных условиях ...,http://manufacturers.ru/company/vtorresurs-0,http://manufacturers.ru/sites/default/files/st...,Лесная промышленность,Лесохимическая промышленность,Переработка макулатуры,2015005799,1172036000290,201501001,"Чеченская Республика, Грозный, улица Фиолетова...",+7(989) 909-89-89,89633963339@mail.ru,http://vtorsyre95.ru,NaN,NaN,6,"Вторресурс, ООО",23.01.2017,NaN
9311,Мичуринскзерностандарт,Мичуринская Мукомольная Компания динамично раз...,http://manufacturers.ru/company/michurinskzern...,http://manufacturers.ru/sites/default/files/st...,Пищевая промышленность,Производство муки и мучных изделий,Мукомольная промышленность,6827015928,1066827009083,682701001,"Тамбовская область, город Мичуринск, Советская...",+74754520075,thmp@mail.ru,http://gkmmk.ru,NaN,NaN,102,"Мичуринскзерностандарт, ООО",24.04.2006,NaN
7903,Клинцовский завод поршневых колец,Открытое акционерное общество «Клинцовский зав...,http://manufacturers.ru/company/klincovskiy-za...,http://manufacturers.ru/sites/default/files/st...,Прочее производство,Производство прочих изделий и о

In [22]:
companies_df.to_csv('companies_large.csv')

In [24]:
companies_df.isna().sum() / len(companies_df)

Название                      0.000000
Описание                      0.000000
Источник                      0.000000
Brand-Image                   0.000000
Отрасль                       0.001270
Категория                     0.001270
Подкатегория                  0.001270
ИНН                           0.083730
ОГРН                          0.124524
КПП                           0.159841
Адрес                         0.000000
Телефоны                      0.013651
Почта                         0.058651
Сайт                          0.005317
Вконтакте                     0.997778
Год основания                 0.973095
Количество сотрудников        0.225476
Название юридического лица    0.119683
Дата регистрации              0.148651
YouTube                       0.999603
dtype: float64

In [ ]:
len(total_dicts)

In [296]:
{'rkir': 'dkfor'}.items()

dict_items([('rkir', 'dkfor')])

In [310]:
c_url = companies_urls[2]
c_info = requests.get(c_url, headers=headers)

In [311]:
soup = BeautifulSoup(c_info.text)

In [312]:
brand_info = soup.find('div', {'class': 'l-brand-block'})

In [321]:
brand_info.find('img')

<img alt="Атекса" height="228" loading="lazy" src="/sites/default/files/styles/logo/public/logo_company/atexa2.jpg?itok=WUNQ1tAn" typeof="foaf:Image" width="228"/>

In [289]:
all_info = soup.find('div', {'class': 'company-right-wrap'})

In [290]:
about = all_info.find(id='about')
name = about.find('h1').text
description = " ".join([p.text for p in about.find_all('p')])


('Атекса',
 'Производство взрывозащищенного электрооборудования — это приоритетное направление развития нашей компании. Основная наша задача — выпуск оборудования, конкурирующего по качеству, стоимости и срокам изготовления, с российскими и зарубежными производителями.\xa0АТЕКСА — это современное производственное предприятие, с инновационной системой управления и станочным парком,\xa0изготавливающее сертифицированное взрывозащищенное оборудование, такое как: взрывозащищенные нагреватели и взрывозащищенные\xa0электрические обогреватели для нефтегазовой отрасли, химической промышленности, атомной промышленности, энергетической промышленности и других отраслей.\xa0Наша компания готова к сотрудничеству и кооперации по любым сложным проектам.')

In [291]:
category_block = all_info.find('div', id='category')
categories = category_block.find_all('tr')
tuples = []
for r in categories:
    tuples.append([td.text.replace("\n", "").strip().replace("  ", "") for td in r.findChildren('td')])
categories = dict((x, y) for x, y in tuples)
categories

{'Отрасль': 'Машиностроение, Нефтегазовая промышленность',
 'Категория': 'Нефтегазовое машиностроение, Обслуживание нефтегазовой отрасли',
 'Подкатегория': 'Производство нефтегазового оборудования, Сервисные услуги'}

In [292]:
requisites_block = all_info.find('div', id='company')
requisites = requisites_block.find_all('tr')
tuples = []
for r in requisites:
    tuples.append([td.text.replace("\n", "").strip() for td in r.findChildren('td')])
requisites = dict((x, y) for x, y in tuples)
requisites

{'Год основания': '2011 г.',
 'Дата регистрации': '08.02.2011',
 'Количество сотрудников': '27',
 'ИНН': '7415071844',
 'ОГРН': '1117415000394',
 'КПП': '741501001',
 'Название юридического лица': 'ООО УралСпецГрупп'}

In [293]:
contacts_block = all_info.find('tbody', {'class': 'cont-tbody'})
contacts = contacts_block.find_all('tr')
tuples = []
for r in contacts:
    tuples.append([td.text.replace("\n", "").strip() for td in r.findChildren('td')])
contacts = dict((x, y) for x, y in tuples)
contacts

{'Адрес': '456313, Челябинская обл., г. Миасс, Тургоякское шоссе 11/63',
 'Телефоны': '8 (800) 333-28-50                    +7 (3513) 29-85-79',
 'Почта': 'zakaz@kalorifer-atexa.ru',
 'Сайт': 'https://kalorifer-atexa.ru/',
 'Вконтакте': 'https://vk.com/atexa'}